In [53]:
# version 1.0
# 这个版本的代码是根据官方提供的思路写的

In [54]:
#导入pandas和aito
import pandas as pd
import numpy as np
from aito.schema import AitoStringType, AitoTextType, AitoDelimiterAnalyzerSchema, AitoTableSchema, AitoColumnLinkSchema, AitoDatabaseSchema
from aito.client import AitoClient
import aito.api as aito_api

In [55]:
#导入各种数据
#商店支付方式
r_payment = pd.read_csv('chefmozaccepts.csv')
#商店烹饪方式
r_cuisine = pd.read_csv('chefmozcuisine.csv')
#商店营业时间
r_business_hours = pd.read_csv('chefmozhours4.csv')
#停车场情况
r_parking_lot = pd.read_csv('chefmozparking.csv')
#商店详细情况
r_geo_places = pd.read_csv('geoplaces2.csv', encoding='latin-1')
#商店评分
r_rating = pd.read_csv('rating_final.csv')
#用户习惯烹饪方式
u_cuisine = pd.read_csv('usercuisine.csv')
#用户习惯支付方式
u_payment = pd.read_csv('userpayment.csv')
#用户概述
u_profile = pd.read_csv('userprofile.csv')

In [56]:
#商店数据处理
#更改商店支付方式列名,并整理数据
r_payment.rename(columns={'Rpayment': 'payment'}, inplace=True)
r_payment = r_payment.groupby('placeID').agg({'payment': lambda x: ';'.join(x)}).reset_index()
#在最开始导入的文件(chefmozaccepts.csv)中,每个placeID可能有多种支付方式
#而每种支付方式均使用了单独一行，所以一个placeID可能占据了多行
#这一步所做的操作就是把重复的placeID的多种支付方式合并到同一行内
#在这一步操作后，每个placeID仅占一行，尽管这个placeID有多种支付方式
#每种支付方式以分号(;)隔开

In [57]:
#这里做的处理和上一个cell道理一样，都是去重复化的处理
#商店停车场
r_parking_lot = r_parking_lot.groupby('placeID').agg({'parking_lot': lambda x: ';'.join(x)}).reset_index()
#商店烹饪方式
r_cuisine.rename(columns={'Rcuisine': 'cuisine'}, inplace=True)
r_cuisine = r_cuisine.groupby('placeID').agg({'cuisine': lambda x: ';'.join(x)}).reset_index()
#商店营业时间
r_business_hours.drop_duplicates(inplace=True)    #去重
r_business_hours.hours = r_business_hours.hours.str.replace(';', '')    #删除hours列里的分号
r_business_hours.replace({'days': {'Mon;Tue;Wed;Thu;Fri;': 'weekdays', 'Sat;': 'saturday', 'Sun;': 'sunday'}}, inplace=True)
#对营业时间进行 one-hot encoding 和 dummy encoding
#对于 one-hot encoding 和 dummy encoding 这两个概念解释起来挺麻烦的
#可以参考这个文章：https://www.jianshu.com/p/5f8782bf15b1
dummies = r_business_hours['days'].str.get_dummies().astype(str)
dummies.values[dummies == '1'] = r_business_hours['hours']
r_business_hours = pd.concat([r_business_hours['placeID'], dummies], axis=1)
#关店时间为close
r_business_hours = r_business_hours.groupby('placeID').agg(lambda values: next((val for val in values if val != '0'), 'closed')).reset_index()
#对于商店详细情况的处理(用NaN替换未知数据"?")
r_geo_places.replace('?', np.nan, inplace=True)
#因为fax和url几乎都是空的，所以去掉这两个数据
r_geo_places.drop(columns=['fax', 'url'], inplace=True)
#将所有数据合成到一个表内，将NaN替换为None
from functools import reduce
places_df = reduce(lambda left,right: pd.merge(left, right, on='placeID', how='outer'), [r_payment, r_cuisine, r_parking_lot, r_business_hours, r_geo_places])
places_df.placeID = places_df.placeID.astype(str)
places_df.replace({np.nan: None}, inplace=True)

In [58]:
#导入用户数据
#用户餐饮习惯
u_cuisine = pd.read_csv('usercuisine.csv')
#用户支付方式习惯
u_payment = pd.read_csv('userpayment.csv')
#用户评分
u_ratings = pd.read_csv('rating_final.csv')
#用户详细资料
u_profile = pd.read_csv('userprofile.csv')

In [59]:
#用户数据处理
#参考上面对商店处理的注释，道理一样
#用户餐饮习惯处理
u_cuisine.rename(columns={'Rcuisine': 'cuisine'}, inplace=True)
u_cuisine = u_cuisine.groupby('userID').agg({'cuisine': lambda x: '; '.join(x)}).reset_index()
#用户支付习惯处理
u_payment.rename(columns={'Upayment': 'payment'}, inplace=True)
u_payment = u_payment.groupby('userID').agg({'payment': lambda x: '; '.join(x)}).reset_index()
#用户评分处理
u_ratings = pd.read_csv('rating_final.csv')
u_ratings.placeID = u_ratings.placeID.astype(str)
#用户资料处理
users_df = reduce(lambda left,right: pd.merge(left, right, on='userID', how='outer'), [u_profile, u_cuisine, u_payment])
users_df.replace({np.nan: None}, inplace=True)

In [60]:
#下面的步骤是Aito处理
#导入pandas的推断表架构
#餐馆数据
places_schema = AitoTableSchema.infer_from_pandas_data_frame(places_df)
for field in ('payment', 'cuisine', 'parking_lot'):
    places_schema[field].data_type = AitoTextType()
    places_schema[field].analyzer = AitoDelimiterAnalyzerSchema(delimiter=';')
for field in ('weekdays', 'saturday', 'sunday'):
    places_schema[field].analyzer = None
    places_schema[field].data_type = AitoStringType()

In [61]:
#用户数据
users_schema = AitoTableSchema.infer_from_pandas_data_frame(users_df)
for field in ('payment', 'cuisine'):
    users_schema[field].data_type = AitoTextType()
    users_schema[field].analyzer = AitoDelimiterAnalyzerSchema(delimiter=';')
#评分数据
ratings_schema = AitoTableSchema.infer_from_pandas_data_frame(u_ratings)

In [62]:
#将评分链接到场所和用户表
ratings_schema['placeID'].link = AitoColumnLinkSchema('places', 'placeID')
ratings_schema['userID'].link = AitoColumnLinkSchema('users', 'userID')
#现在我们有了数据库模式：
db_schema = AitoDatabaseSchema(tables={'users': users_schema, 'places': places_schema, 'ratings': ratings_schema})

In [63]:
#设置凭证(credentials)并将数据上传到Aito
AITO_INSTANCE_URL = 'https://kalinote.aito.app'
AITO_API_KEY = 'jh7y40WzpZ7nFsph7Lkpy49DkBoO5Ivh41UpjObG'

In [64]:
client = AitoClient(instance_url=AITO_INSTANCE_URL, api_key=AITO_API_KEY)
#aito_api.create_database(client=client, schema=db_schema)
#aito_api.upload_entries(client=client, table_name='places', entries=places_df.to_dict(orient='records'))
#aito_api.upload_entries(client=client, table_name='users', entries=users_df.to_dict(orient='records'))
#aito_api.upload_entries(client=client, table_name='ratings', entries=u_ratings.to_dict(orient='records'))

In [65]:
#查询
query = {
    "from": "places",
    "where": {
        "price": "low",
        "payment": "cash",
    }
}
res = aito_api.generic_query(client=client, query=query)
print(res.to_json_string(indent=2))

{
  "offset": 0,
  "total": 28,
  "hits": [
    {
      "Rambience": "familiar",
      "accessibility": "no_accessibility",
      "address": "frente al tecnologico",
      "alcohol": "No_Alcohol_Served",
      "area": "open",
      "city": "victoria",
      "country": "mexico",
      "cuisine": "Regional",
      "dress_code": "informal",
      "franchise": "f",
      "latitude": 23.752304100000003,
      "longitude": -99.16691329999999,
      "name": "puesto de gorditas",
      "other_services": "none",
      "parking_lot": "public",
      "payment": "cash",
      "placeID": "132560",
      "price": "low",
      "saturday": "00:00-00:00",
      "smoking_area": "permitted",
      "state": "tamaulipas",
      "sunday": "00:00-00:00",
      "the_geom_meter": "0101000020957F0000FC60BDA8E88157C1B2C357D6DA4E4941",
      "weekdays": "08:00-12:00"
    },
    {
      "Rambience": "familiar",
      "accessibility": "completely",
      "alcohol": "No_Alcohol_Served",
      "area": "closed",
     

In [68]:
#推荐
rec_query = {
    "from": "ratings",
    "where": {
        "userID": {
            "cuisine": "Mexican", 
            "payment": "VISA"
        }
    },
    "recommend": "placeID",
    "goal": {"rating": 2}
}
res = aito_api.recommend(client=client, query=rec_query)
print(res.top_recommendation)

{"$p": 0.9733343347281155, "Rambience": "familiar", "accessibility": "no_accessibility", "address": "Ricardo Linares 107", "alcohol": "Wine-Beer", "area": "closed", "city": "Cuernavaca", "country": "Mexico", "cuisine": "International", "dress_code": "formal", "franchise": "f", "latitude": 18.928798, "longitude": -99.239513, "name": "Restaurant Las Mananitas", "other_services": "none", "parking_lot": "yes", "placeID": "134986", "price": "high", "saturday": "00:00-23:30", "smoking_area": "none", "state": "Morelos", "sunday": "00:00-23:30", "the_geom_meter": "0101000020957F00002A0D05E2D96D5AC1AB058CB1EC564A41", "weekdays": "00:00-23:30"}
